In [1]:
import pandas as pd
import os
from glob import glob

In [2]:
test_filepath = os.path.join('C:\DATA\ComBERT\data_finetuning_spamFiltering', 'test_containing_company_name_1000.csv')

### SEC-BERT without post-training vs SEC-BERT with the proposed company name masking method

In [3]:
df = pd.read_csv(test_filepath)
df
# finetune_size = 50
# model_name = 'SECBERT'
# method_name
# df['ft{}_{}_{}_seed{}'.format(finetune_size, model_name, method_name, seed_num)] = df['text'].apply(lambda x: )

,raw_text,is_retweet,label,created_at,yyyymm,text,contains_company_name
0,RT @BursatilBiotech: Dont forget about these c...,True,human,2017-09-06 04:42:15,201709,[rt] dont forget about these couple of upcomin...,True
1,RT @CryptoHedge_: We're LIVE with Jack Mallers...,True,human,2017-09-02 18:42:04,201709,rt _: we re live with jack mallers creator of ...,True
2,GM executives think that $GM stock is a strong...,False,human,2017-09-02 12:53:15,201709,gm executives think that general motors co sto...,True
3,$MRNS ↑ 65%,False,human,2017-09-11 13:30:53,201709,"marinus pharmaceuticals, inc. ↑ 65",True
4,"Don't get me wrong, I see $ltc at $5???? wow",False,human,2017-09-07 15:45:26,201709,don t get me wrong i see ltc properties inc at...,True
...,...,...,...,...,...,...,...
995,"RT @gkhnozkan: time to pump $snt, 900 to min 2...",True,bot,2017-09-03 18:49:53,201709,[rt] time to pump senstar technologies ltd. 90...,True
996,RT @APACllc: Thanks dude. $btc $mtl $omg $neo ...,True,bot,2017-09-08 00:42:31,201709,[rt] thanks dude $btc mechel pao $omg neogenom...,True
997,RT @AltTradlngClub: WANNA MAKE HUGE PROFIT????...,True,bot,2017-09-04 08:53:07,201709,[rt] wanna make huge profit join us [url] adt ...,True
998,RT @TickerReport: Whitebox Advisors Llc Sells ...,True,bot,2017-09-01 11:07:38,201709,[rt] whitebox advisors llc sells 20 900 shares...,True


In [4]:
target_trained_with_nums = [50, 100, 200, 300, 400]
filepaths_format = os.path.join('C:\DATA\ComBERT', 'results_spamFiltering', 'testCompanyName=y_SECBERT_{}_*', 'results_train_{}.csv')

In [5]:
target_trained_with_num = target_trained_with_nums[0]

method_name = 'CM'
filepaths = glob(filepaths_format.format(method_name, target_trained_with_num))
print(target_trained_with_num, method_name)
print('Number of filepaths: {}'.format(len(filepaths)))

dfs = []
for filepath in filepaths:
    one_df = pd.read_csv(filepath)
    one_df = one_df[(one_df['text'].str.contains('amazon'))&(one_df['true_label']==1)&(one_df['correct'] == True)]
    one_df['method_name'] = method_name
    one_df['data_size'] = target_trained_with_num
    one_df['seed_num'] = os.path.dirname(filepath)[-1]
    dfs.append(one_df)
result = pd.concat(dfs)

result

50 CM
Number of filepaths: 5


,text,true_label,predicted_label,correct,method_name,data_size,seed_num
510,[rt] just another manic monday hurricane relie...,1,1,True,CM,50,1
529,[rt] just another manic monday hurricane relie...,1,1,True,CM,50,1
654,[rt] [url] is hunting for a city for a $5 bill...,1,1,True,CM,50,1
807,rt _insider: amazon amazon com inc could be wo...,1,1,True,CM,50,1
809,[rt] may was one of my favorite months of trad...,1,1,True,CM,50,1
895,[rt] hurricane irma rocks the stock market spd...,1,1,True,CM,50,1
510,[rt] just another manic monday hurricane relie...,1,1,True,CM,50,2
654,[rt] [url] is hunting for a city for a $5 bill...,1,1,True,CM,50,2
895,[rt] hurricane irma rocks the stock market spd...,1,1,True,CM,50,2
654,[rt] [url] is hunting for a city for a $5 bill...,1,1,True,CM,50,3


In [6]:
dfs = [result]
for text in result['text'].values:
    for method_name in ['NoPT', 'SM', 'WWM']:
        filepaths = glob(filepaths_format.format(method_name, target_trained_with_num))

        for filepath in filepaths:
            one_df = pd.read_csv(filepath)
            one_df = one_df[(one_df['text']==text)&(one_df['correct'] == False)]
            one_df['method_name'] = method_name
            one_df['data_size'] = target_trained_with_num
            one_df['seed_num'] = os.path.dirname(filepath)[-1]
            dfs.append(one_df)
result = pd.concat(dfs)
result

,text,true_label,predicted_label,correct,method_name,data_size,seed_num
510,[rt] just another manic monday hurricane relie...,1,1,True,CM,50,1
529,[rt] just another manic monday hurricane relie...,1,1,True,CM,50,1
654,[rt] [url] is hunting for a city for a $5 bill...,1,1,True,CM,50,1
807,rt _insider: amazon amazon com inc could be wo...,1,1,True,CM,50,1
809,[rt] may was one of my favorite months of trad...,1,1,True,CM,50,1
...,...,...,...,...,...,...,...
807,rt _insider: amazon amazon com inc could be wo...,1,0,False,WWM,50,2
895,[rt] hurricane irma rocks the stock market spd...,1,0,False,NoPT,50,0
895,[rt] hurricane irma rocks the stock market spd...,1,0,False,NoPT,50,3
895,[rt] hurricane irma rocks the stock market spd...,1,0,False,SM,50,0


In [14]:
print(len(result['text'].unique()))
print(result['text'].unique())

target_text = list(result['text'].unique())[0]
print(target_text)

6
['[rt] may was one of my favorite months of trading here s why: ferrari n.v. $spxu sbux amazon com inc univar solutions inc. baba $study #finance #trading http'
 '[rt] hurricane irma rocks the stock market spdr s&p 500 etf trust amazon com inc #dollar $xrt #tgif [url] [url]'
 '[rt] just another manic monday hurricane relief lowes companies inc $spx $dxd amazon com inc $dji $kie valero energy corp/tx barclays bank plc united states natural gas fund, lp apple inc. [url]'
 '[rt] [url] is hunting for a city for a $5 billion 50 000 worker 2nd headquarters amazon com inc'
 'rt _insider: amazon amazon com inc could be worth another 3 5x in 7 8 years mkm [url]'
 '[rt] just another manic monday hurricane relief spdr dow jones industrial average etf trust spdr s&p 500 etf trust amazon com inc #irma [url] [url]']
[rt] may was one of my favorite months of trading here s why: ferrari n.v. $spxu sbux amazon com inc univar solutions inc. baba $study #finance #trading http


In [15]:
result[(result['text']==target_text)&(result['method_name']=='NoPT')]

,text,true_label,predicted_label,correct,method_name,data_size,seed_num
809,[rt] may was one of my favorite months of trad...,1,0,False,NoPT,50,0
809,[rt] may was one of my favorite months of trad...,1,0,False,NoPT,50,2
809,[rt] may was one of my favorite months of trad...,1,0,False,NoPT,50,3
809,[rt] may was one of my favorite months of trad...,1,0,False,NoPT,50,4


In [16]:
result[(result['text']==target_text)&(result['method_name']=='SM')]

,text,true_label,predicted_label,correct,method_name,data_size,seed_num
809,[rt] may was one of my favorite months of trad...,1,0,False,SM,50,0
809,[rt] may was one of my favorite months of trad...,1,0,False,SM,50,3


In [17]:
result[(result['text']==target_text)&(result['method_name']=='WWM')]

,text,true_label,predicted_label,correct,method_name,data_size,seed_num
809,[rt] may was one of my favorite months of trad...,1,0,False,WWM,50,0


In [18]:
df[df['text']==target_text].iloc[0]['raw_text']

"RT @SharePlanner: May was one of my favorite months of trading -Here's why: $RACE $SPXU SBUX $AMZN $UNVR BABA $STUDY #finance #trading http…"